## 🚀 Quick Start Note

**✅ The Streamlit app now automatically processes data!** 

Just run `streamlit run app/app.py` directly - no need to execute this notebook first.

### 📚 Why This Notebook Still Exists:

- **🔍 Transparency**: See exactly how your data is processed
- **🛠️ Customization**: Modify ETL logic for different data sources  
- **📊 Exploration**: Interactive analysis and debugging
- **🎓 Learning**: Understand data science best practices
- **🔧 Flexibility**: Alternative workflow for notebook-based users

### 🎯 When to Use This Notebook:

- You want to understand the data processing pipeline
- You need to modify the ETL logic
- You prefer interactive data exploration
- You want to add new data sources or metrics
- You're learning about data engineering workflows

# Data Processing Pipeline (Optional)

**📱 For App Users**: This processing now happens automatically in the Streamlit app!

**🔬 For Data Scientists**: This notebook shows the complete ETL pipeline for transparency and customization.

This notebook demonstrates how marketing and business data is processed into a unified daily dataset for the dashboard.

## 1. Import Required Libraries
Import pandas and numpy for data manipulation and analysis.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
import zipfile
import os

## 2. Load CSV Files and Parse Dates
Load Facebook.csv, Google.csv, TikTok.csv, and Business.csv from the data/ directory, ensuring date columns are parsed as datetime objects.

In [3]:
# Define data directory
DATA_DIR = Path('../data')

# Load CSVs with date parsing
df_fb = pd.read_csv(DATA_DIR / 'Facebook.csv', parse_dates=['date'])
df_gg = pd.read_csv(DATA_DIR / 'Google.csv', parse_dates=['date'])
df_tt = pd.read_csv(DATA_DIR / 'TikTok.csv', parse_dates=['date'])
df_biz = pd.read_csv(DATA_DIR / 'Business.csv', parse_dates=['date'])

In [4]:
# Unzip any zip files in the data directory
for file in os.listdir(DATA_DIR):
    if file.endswith('.zip'):
        with zipfile.ZipFile(DATA_DIR / file, 'r') as zip_ref:
            zip_ref.extractall(DATA_DIR)
        print(f"Unzipped {file}")

## 3. Standardize Column Names
Rename columns in all DataFrames to a consistent naming convention (e.g., lowercase, underscores).

In [5]:
def standardize_columns(df):
    df.columns = (
        df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('#', 'num').str.replace(r'[^a-z0-9_]', '', regex=True)
    )
    return df

# Standardize columns
df_fb = standardize_columns(df_fb)
df_gg = standardize_columns(df_gg)
df_tt = standardize_columns(df_tt)
df_biz = standardize_columns(df_biz)

# Rename columns to match expected names for aggregation
def rename_marketing_cols(df):
    return df.rename(columns={
        'impression': 'impressions',
        'attributed_revenue': 'revenue'
    })

df_fb = rename_marketing_cols(df_fb)
df_gg = rename_marketing_cols(df_gg)
df_tt = rename_marketing_cols(df_tt)

# Rename business columns for consistency
df_biz = df_biz.rename(columns={
    'total_revenue': 'business_total_revenue',
    'gross_profit': 'business_gross_profit',
    'num_of_orders': 'business_orders',
    'num_of_new_orders': 'business_new_orders',
    'new_customers': 'business_new_customers',
    'cogs': 'business_cogs'
})

## 4. Aggregate Marketing Spend and Revenue by Date
Group Facebook, Google, and TikTok data by date, summing spend, impressions, clicks, and revenue for each day.

In [6]:
# Aggregate by date for each channel
def agg_channel(df, channel):
    agg = df.groupby('date').agg({
        'spend': 'sum',
        'impressions': 'sum',
        'clicks': 'sum',
        'revenue': 'sum'
    }).rename(lambda x: f"{channel}_" + x, axis=1)
    return agg

agg_fb = agg_channel(df_fb, 'fb')
agg_gg = agg_channel(df_gg, 'gg')
agg_tt = agg_channel(df_tt, 'tt')

# Merge all marketing data on date
df_marketing = agg_fb.join([agg_gg, agg_tt], how='outer')

## 5. Join Business Performance Data
Merge the aggregated marketing data with the business performance data on the date column.

In [7]:
# Set date as index for business data
df_biz_idx = df_biz.set_index('date')

# Merge with marketing data
df_merged = df_marketing.join(df_biz_idx, how='outer')

## 6. Handle Missing Dates and Fill Gaps
Create a complete date range covering all dates in the dataset, reindex the merged DataFrame, and fill missing values appropriately.

In [8]:
# Create a complete date range
full_range = pd.date_range(df_merged.index.min(), df_merged.index.max(), freq='D')
df_merged = df_merged.reindex(full_range)
df_merged.index.name = 'date'

# Fill missing values: 0 for spend/clicks/impressions/revenue, ffill for business KPIs
for col in df_merged.columns:
    if any(x in col for x in ['spend', 'clicks', 'impressions', 'revenue']):
        df_merged[col] = df_merged[col].fillna(0)
    else:
        df_merged[col] = df_merged[col].ffill()

## 7. Derive CTR, CPC, and ROAS Metrics
Calculate Click-Through Rate (CTR), Cost Per Click (CPC), and Return on Ad Spend (ROAS) for each day.

In [9]:
# Calculate total spend, clicks, impressions, revenue per day
df_merged['total_spend'] = df_merged[[c for c in df_merged.columns if 'spend' in c]].sum(axis=1)
df_merged['total_clicks'] = df_merged[[c for c in df_merged.columns if 'clicks' in c]].sum(axis=1)
df_merged['total_impressions'] = df_merged[[c for c in df_merged.columns if 'impressions' in c]].sum(axis=1)
df_merged['total_revenue'] = df_merged[[c for c in df_merged.columns if 'revenue' in c and not c.startswith('business')]].sum(axis=1)

# CTR: Clicks / Impressions
# CPC: Spend / Clicks
# ROAS: Revenue / Spend
df_merged['ctr'] = np.where(df_merged['total_impressions'] > 0, df_merged['total_clicks'] / df_merged['total_impressions'], np.nan)
df_merged['cpc'] = np.where(df_merged['total_clicks'] > 0, df_merged['total_spend'] / df_merged['total_clicks'], np.nan)
df_merged['roas'] = np.where(df_merged['total_spend'] > 0, df_merged['total_revenue'] / df_merged['total_spend'], np.nan)

In [10]:
# The df_merged DataFrame is now a daily panel with all metrics
panel_df = df_merged.copy()
panel_df.head()

,fb_spend,fb_impressions,fb_clicks,fb_revenue,gg_spend,gg_impressions,gg_clicks,gg_revenue,tt_spend,tt_impressions,...,business_total_revenue,business_gross_profit,business_cogs,total_spend,total_clicks,total_impressions,total_revenue,ctr,cpc,roas
date,,,,,,,,,,,,,,,,,,,,,
2025-05-16,15440.67,1475903,19638,40775.59,12669.47,2033462,83448,36422.52,11997.12,1432113,...,238010.95,124434.71,113576.24,40107.26,125323,4941478,110281.85,0.025361,0.320031,2.749673
2025-05-17,17210.84,1677399,21777,42695.57,13250.04,1905351,79998,39544.91,11222.08,1462304,...,224086.40,126119.02,97967.38,41682.96,124233,5045054,114401.63,0.024625,0.335522,2.744566
2025-05-18,17492.64,1687499,23297,46380.88,12191.80,1880528,76834,40701.78,13274.73,1689160,...,226582.57,128282.08,98300.49,42959.17,125364,5257187,123677.58,0.023846,0.342675,2.878956
2025-05-19,16296.34,1600554,21285,42482.04,12085.40,1679868,70965,37043.71,11556.30,1487766,...,203540.86,100212.05,103328.81,39938.04,115391,4768188,111469.36,0.024200,0.346111,2.791057
2025-05-20,14728.15,1642554,21760,39414.45,8239.23,1846924,68390,23916.80,10250.38,1321766,...,188081.24,115962.11,72119.13,33217.76,109106,4811244,92178.22,0.022677,0.304454,2.774968


## 9. Export Final DataFrame to df_final.csv
Save the final daily panel DataFrame as df_final.csv in the data/ directory.

In [11]:
# Export to CSV
panel_df.to_csv(DATA_DIR / 'df_final.csv')
print('Exported df_final.csv to data/')

Exported df_final.csv to data/


In [12]:
# Export to CSV
panel_df.to_csv(DATA_DIR / 'df_final.csv')
print('Exported df_final.csv to data/')

Exported df_final.csv to data/


## 8. Merge All Data into Daily Panel DataFrame
Combine all processed data into a single daily panel DataFrame, ensuring all metrics are included.